# Install Spark

In [ ]:
#If you are using  Data proc comment the file for spark instalattion.
# !sudo apt update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# #Update spark version for install
# !wget -q  https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# !tar xf spark-3.3.2-bin-hadoop3.tgz

# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"


# !pip install -q findspark
# !pip install pyspark

# import findspark
# findspark.init()
# findspark.find()

from pyspark.sql import DataFrame, SparkSession



spark = SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

spark

In [ ]:
spark = SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()
spark

# Download the file

In [ ]:
import urllib
url = 'https://drive.google.com/file/d/1BFx9CClKTe_syBuIr5WBi5uuSkZf9wdv/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
urllib.request.urlretrieve(url, "people-with-dups.csv")


# Instructions

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) De-Duping Data Lab

In this exercise, we're doing ETL on a file we've received from a customer. That file contains data about people, including:

* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL").
* The Social Security numbers aren't consistent either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. The salaries will match as well,
and the Social Security Numbers *would* match if they were somehow put in the same format.

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Standarize the field ssn with out hyphens.
* Create a colum age base on the birth date.


Finally, you will write the results as a Parquet file.


**Hint:** The initial dataset contains 10,000 records.<br/>
The de-duplicated result has 9,976 records.


# Read the file

In [ ]:
#load the file into HDFS system
# Run this just is are using a data proc cluster
!hdfs dfs -put people-with-dups.csv /user/root/people-with-dups.csv
!hdfs dfs -ls /user/root/

In [ ]:
#Read data from csv file
#Optional read the csv file from a gcp bucket
from pyspark.sql.functions import *
path = "people-with-dups.csv"

df = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(path))

#see just 10 record of the data set
df.show(10)
 

# Processing
You can use the [SQL API reference](https://hyukjin-spark.readthedocs.io/en/latest/reference/pyspark.sql.html) for help

In [ ]:
#Create new name columns with only lower case names
dedupedDF = df.withColumn('firstName_lower', lower(col('firstName'))) \
          .withColumn('middleName_lower', lower(col('middleName'))) \
          .withColumn('lastName_lower', lower(col('lastName')))

#Create new ssn column without any dashes

#Create a new age field

#Drop duplicates based only on the new columns, gender, birthDate and salary

#Drop the old columns



#See everything is ok with the final df
dedupedDF.show(5)



In [ ]:
#See the number of rows


# Save the file

In [ ]:
#Write the processed data frame on parquet format
#Optional Write the csv file in a gcp bucket
path = "people-deduped.parquet"


In [ ]:
#validate if the parque file was writed in the HDFS
!hdfs dfs -ls /user/root/people_without_dups

# Analytics process

In [ ]:
# Instructions
Add the code in Spark to do the below requirements.
How much the process of deduplication affect the Average Salary (do a difference before and after cleaning the data)

There is suspicion this data was simulated, What is the Average Salary grouping in ages with 10-year intervals (0-10, 10-20, 20-30), just for people from 20 to 60 years old. 

Finally, you will write the results as a Parquet file.

In [ ]:
# Get the average Salary

In [ ]:
#Get Average Salary grouping

# Check your answer

In [ ]:
finalCount = dedupedDF.count()
assert finalCount == 9976, "expected 9976 records in finalDF"